# LLM Agents
This is the third notebook for the LLM section of Comp 255.

1. [Is RAG an agent?](#is-rag-an-agent)
2. [Tool use with AgentBot](#tool-use-with-agentbot)
   - [Exercise: Make a RAG tool](#exercise-make-a-rag-tool)
3. Fine-tuning: Access [this notebook](https://colab.research.google.com/drive/1HB5c6RkyOljXXghih11Y8Xbe51uhO2s2?usp=sharing) for this section

In [ ]:
from llamabot import QueryBot, AgentBot, tool
from pathlib import Path

sft_model = "qwen2.5:1.5b"
agent_model = "qwen2.5:7b"

## Is RAG an agent?
Let's set up our RAG workflow from last time and consider whether this meets the definition of "agent"

In [ ]:
# setting the data back up
doc1 = "Languages with statutory recognition in Papua New Guinea \
are Tok Pisin, English, Hiri Motu, and \
Papua New Guinean Sign Language"

doc2 = "The only language with statutory recognition in France \
is French"

# write these to a temporary file
with open("doc1.txt", "w") as f:
    f.write(doc1)

with open("doc2.txt", "w") as f:
    f.write(doc2)

In [ ]:
system_message = "You are a helpful assistant that answers questions \
    based on the provided documents."
doc_paths = [Path("doc1.txt"), Path("doc2.txt")]

query_completer = QueryBot(
    system_prompt=system_message,
    model_name = f"ollama_chat/{sft_model}",
    collection_name="documents",
    document_paths=doc_paths
)

In [ ]:
response = query_completer('Hi how are you today!')


Is this an agent? It did respond reasonably, but it seems to be pretty preoccupied with the RAG use-case.  An agent should have "agency"; it should be able to decide the right action to take.  Sometimes that might involve the RAG workflow, but other times it won't.

## Tool use with AgentBot

Just like you would use Google to figure out what languages are spoken in a particular country, the agent can be designed to use "tools" to answer a question.  In that case, the retrieval workflow above could just be considered a tool.

A tool, in this case, is essentially just a function or set of functions.  Since the LLM can't directly run a function, it has to generate a response that can then be parsed into what function to call and what parameters to use.

Here we create a simple tool for calculating the total of a restaurant bill with tip.  Then we create an `AgentBot` instance, which uses a specially-designed prompt with information about the available tools.  With this information, the LLM can decide whether or not to use a tool and, if it uses a tool, what parameters to supply.

NOTE: This is a COMPLEX task and requires a bigger model than we've been using thus far.  Complex workflows are likely outside the capabilities of even this larger model.

In [ ]:
# adapted from https://github.com/ericmjl/llamabot/blob/main/docs/tutorials/agentbot.md
# llamabot.tool decorator
@tool
def calculate_total_with_tip(bill_amount: float, tip_rate: float) -> float:
    if tip_rate < 0 or tip_rate > 1.0:
        raise ValueError("Tip rate must be between 0 and 1.0")
    return bill_amount * (1 + tip_rate)

# Create the bot
bot = AgentBot(
    system_prompt="You are my assistant with respect to restaurant bills.",
    functions=[calculate_total_with_tip, ],
    model_name=f"ollama_chat/{agent_model}",
    #model_name='gpt-4o'  
)

In [ ]:
# look at what is being provided to the model
print(calculate_total_with_tip.json_schema)

In [ ]:
bot.tools

In [ ]:
# system prompt has been extended with additional information
bot.decision_bot.system_prompt.content

In [ ]:
calculate_total_only_prompt = "My dinner was $2300 without tips. Calculate my total with an 18% tip."
response = bot(calculate_total_only_prompt, max_iterations=4)
print(response.content)

This will *usually* work.  The issue with small models (7 billion parameters in this case) is that they can sometimes output strange things, which will cause errors.  Sometimes the models can self-correct, but sometimes not.

The bot now could be said to have "agency"; it can decide whether to use a tool or just go ahead and respond normally.

In [ ]:
response = bot('Hello how are you?')
print(response.content)

### Exercise: Make a RAG tool
Since you already have the syntax above, you can wrap our previous retrieval workflow in a tool function.  Try it out.

In [ ]:
# a bot within a bot - you can likely replace this with the LanceDB query itself
system_message = "You are a helpful assistant that can answer questions \
    based on the provided documents."
doc_paths = [Path("doc1.txt"), Path("doc2.txt")]
query_completer = QueryBot(
    system_prompt=system_message,
    model_name=f"ollama_chat/{agent_model}",
    collection_name="documents",
    document_paths=doc_paths,
)

@tool
def information_retrieval(query: str) -> str:
    query_completer(query)

# Create the bot
bot = AgentBot(
    system_prompt="You are my assistant and can retrieve information if needed.",
    functions=[information_retrieval],
    model_name=f"ollama_chat/{sft_model}",   
)

In [ ]:
response = bot("Hello how are you?")
print(response.content)

In [ ]:
response = bot("What are the national languages of Papua New Guinea")
print(response.content)

Again, we see some oddities, but we've managed to roughly get our RAG workflow to be an tool that the agent is able to make a decision to call with specific parameters.